### Notes
If the error message comes back as "SessionNotCreatedException: Message: session not created: 
This version of ChromeDriver (OperaDriver) only supports Chrome (Opera) version xx",
get the latest Chrome (Opera) webdriver from

https://chromedriver.chromium.org/downloads (https://www.selenium.dev/downloads/)

or from github here

https://github.com/GoogleChrome/chrome-launcher/releases (https://github.com/operasoftware/operachromiumdriver/releases)

and then save it here

C:/SeleniumDrivers/

In [1]:
#!pip install webdriver_manager

In [2]:
# create a time difference function
import datetime

def timediff(start, end, decimals = 1):
    if   int((end - start) / 3600) > 0: # non-zero hours
        return str(int(  (end - start) / 3600)) + 'hr '  + str(int(  (end - start) / 60)          ) + 'min '\
         +                                                 str(round((end - start) % 60, decimals)) + 'sec'
    elif int((end - start) / 60)   > 0: # non-zero hours and minutes
        return str(int(  (end - start) / 60))   + 'min ' + str(round((end - start) % 60, decimals)) + 'sec'
    else:
        return                                             str(round((end - start) % 60, decimals)) + 'sec'

In [3]:
# load libraries
import pandas as pd
import numpy as np
import os
import csv
import datetime
from datetime import date, datetime
#from selenium.webdriver.opera.options import Options # to state absolute path of the Opera Browser
#from selenium.webdriver.chrome.options import Options # to state absolute path of the Chrome browser
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#import networkdays as networkdays

In [4]:
# set paths to the driver, urls, and report paramters

os.environ["PATH"] = r'C:/SeleniumDrivers' # + os.pathsep + os.getenv("PATH")
# https://stackoverflow.com/questions/61213005/modify-beginning-of-path-variable-with-os-environ-in-python
eagle_portal       = r'https://eagleportal.prescient.co.za'
url_default        = eagle_portal + r'/Default.aspx'
url_derv           = eagle_portal + r'/Queries/Query.aspx?rpt=DerivativeExposure'
url_parn           = eagle_portal + r'/Queries/Query.aspx?rpt=PortfolioAnalytics'

pth                = r'P:\Investment Operations\GRC\Compliance\Daily\derv_report.xlsx' # variables stored here

In [5]:
# set variables
derv_report = pd.read_excel(pth, sheet_name = 'Report', usecols = 'B', nrows = 2)
tbl_fnds_   = pd.read_excel(pth, sheet_name = 'Report', usecols = 'A').dropna()
flatfnds_   = [element for innerList in tbl_fnds_.values for element in innerList] # https://www.geeksforgeeks.org/python-convert-a-nested-list-into-a-flat-list/
fnds_       = (',').join(flatfnds_) # https://stackoverflow.com/questions/12453580/how-to-concatenate-join-items-in-a-list-to-a-single-string

aladdin     = derv_report.iat[0,0]
sesame      = derv_report.iat[1,0]
when        = list(derv_report.columns)[0]
when_fmt    = when.strftime('%#m/%#d/%Y') #https://stackabuse.com/how-to-format-dates-in-python/

day_        = f'{when:%#d}' # f'{when:%d}' report date with leading zeroes
month_year_ = f'{when:%B}, {when:%Y}'
month_      = f'{when:%b}'
year_       = f'{when:%Y}'

print(when, '\n', when_fmt, '\n',  fnds_)

2023-08-30 00:00:00 
 8/30/2023 
 PLPRNA,PLMED,3BBCIINC,ABMMABR,ABMMBND,ABMMEQU


In [6]:
from selenium import webdriver
# Firewfox (x = 1),  Opera (x = 2), or Chrome (x = 3)
x = 1 # select Firefox webdriver

if   x == 1:    #https://stackoverflow.com/questions/14603478/firefox-wont-drive-with-selenium-webdriver
    driver = webdriver.Firefox()
    driver.get(url_parn)

elif x == 2:
    from selenium.webdriver.chrome.service import Service as ChromeService
    from webdriver_manager.chrome import ChromeDriverManager
    from selenium.webdriver.chrome.options import Options # to state absolute path of the Chrome browser
    driver = webdriver.Chrome()
    #driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    print(driver.capabilities['chrome']['chromedriverVersion'].split(' ')[0])
    
elif x == 3:
    from selenium.webdriver.common import desired_capabilities
    #from selenium.webdriver.opera.options import Options # to state absolute path of the Opera Browser
    driver = webdriver.Chrome(r'C:\SeleniumDrivers\operadriver.exe')
    print(driver.capabilities['browserVersion'])

In [7]:
# open the browser on the Eagle web page
driver.get(url_parn)             # fund holdings page
wait = WebDriverWait(driver, 10) # https://selenium-python.readthedocs.io/waits.html, max wait for elements to appear

In [8]:
# login with credentials
driver.find_element(By.CSS_SELECTOR, '#LoginCtrl_MainLoginControl_UserName'   ).send_keys(aladdin)
driver.find_element(By.CSS_SELECTOR, '#LoginCtrl_MainLoginControl_Password'   ).send_keys(sesame)
driver.find_element(By.CSS_SELECTOR, '#LoginCtrl_MainLoginControl_LoginButton').click()

# having logged in, switch to the query page
driver.find_element(By.CSS_SELECTOR, '#ModifyLinkLabel'                       ).click()

edit_criteria_window = driver.window_handles[0] # save curent window handle
# How to switch to new window in Selenium for Python?
# https://stackoverflow.com/questions/10629815/how-to-switch-to-new-window-in-selenium-for-python

In [9]:
cal_wait  = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_I"]')))
cal_wait.click()
cal_input = driver.find_element(                   By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_I"]')
driver.execute_script(f'arguments[0].value = "{when_fmt}";', cal_input)

In [11]:
# open the calendar
cal_popup       = driver.find_element(By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_B-1"]')
cal_popup.click()

# get the comma separator between month and date. Avoids the default 'month year' without a separating comma
#wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_NMC"]'))).click() # (right) shift calendar to next month
#wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_PMC"]'))).click() #  (left) shift calendar to previous month

# set up the calendar left-clicker
lmonth_selector = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_PMC"]')))

TimeoutException: Message: 


In [ ]:
from_mnth_selector = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_I"]')))
to_month_selector  = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_To_I"]'  )))

In [34]:
# left click the calendar until the reporting 'month year' combination presents
while driver.find_element(By.XPATH,'//td[@id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_TC"]').text != month_year_:
    lmonth_selector.click()
# use Python Selenium to get span text; https://stackoverflow.com/questions/14590341/use-python-selenium-to-get-span-text
    
# select the reporting day on the calendar
day_selector = driver.find_element(By.XPATH,f'//td[@class="dxeCalendarDay"][text()={day_}] | //td[@class="dxeCalendarDay dxeCalendarWeekend"][text()={day_}]')
day_selector.click()

In [35]:
# get the web elements of fund list, and of the submit button
fund_selector  = driver.find_element(By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SelectedIds"]')

# assign values to the fund list element
driver.execute_script(f'arguments[0].value = "{fnds_}";', fund_selector)

In [36]:
# click the 'Submit' button
submit_button  = driver.find_element(By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_RunBtn"]')
submit_button.click()

In [26]:
xls_button  = driver.find_element(By.CSS_SELECTOR, 'td[id="ExportMnu_DXI4_T"]')
xls_button.click()

ElementNotInteractableException: Message: Element <td id="ExportMnu_DXI4_T" class="item"> could not be scrolled into view
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:187:5
ElementNotInteractableError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:347:5
webdriverClickElement@chrome://remote/content/marionette/interaction.sys.mjs:152:11
interaction.clickElement@chrome://remote/content/marionette/interaction.sys.mjs:121:11
clickElement@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:204:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.sys.mjs:86:31


In [ ]:
# SPARE
# click the month, year selector
month_year_selector = driver.find_element(By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_DDD_C_TC"]')
month_year_selector.click()

In [ ]:
pre_calendar = driver.window_handles[0]
pre_calendar

In [ ]:
curr_calendar = driver.window_handles[0]
curr_calendar

In [ ]:
driver.find_element(By.XPATH,'//td[@class="dxeCalendarFastNavYear dxeCalendarFastNavYearSelected"]').text

In [ ]:
# SPARE - open the calendar
#cal_popup       = driver.find_element(By.CSS_SELECTOR, 'td[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_DATE1_DateCtrl_From_B-1"]')
#cal_popup.click()

In [ ]:
driver.find_element(By.XPATH,'//td[@class="dxeCalendarFastNavYearSelected"]').text

In [ ]:
# get the web elements of fund list, and of the submit button
fund_selector  = driver.find_element(By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SelectedIds"]')

# assign values to the fund list elements
driver.execute_script(f'arguments[0].value = "IJGIPF,QIFFGIF";', fund_selector)

In [ ]:
# click the submit button


In [ ]:
list(driver.find_elements(By.XPATH,'//td[@class="dxeCalendarFastNavYear"] | td[@class="dxeCalendarFastNavYear dxeCalendarFastNavYearSelected"]'))

In [ ]:
list(e.text for e in driver.find_elements(By.XPATH,'//td[@class="dxeCalendarFastNavYear"] | td[@class="dxeCalendarFastNavYear dxeCalendarFastNavYearSelected"] | td[@class="dxeCalendarFastNavYear dxeCalendarFastNavYearSelected dxeCalendarFastNavYearHover"]'))

In [ ]:
x = driver.find_elements(By.XPATH,'//td[@class="dxeCalendarFastNavYear"] | td[@class="dxeCalendarFastNavYear dxeCalendarFastNavYearSelected"]')
x

In [ ]:
# find the year shift-left clicker button
lyear_selector = driver.find_element(By.CSS_SELECTOR, 'img[class="dxEditors_edtCalendarFNPrevYear"]') # year shift-left selector

# find the correct year
while year_ not in list(e.text for e in driver.find_elements(By.XPATH,'//td[@class="dxeCalendarFastNavYear"] | td[@class="dxeCalendarFastNavYear dxeCalendarFastNavYearSelected"]')):
    lyear_selector.click()
    
# click the correct year

In [ ]:
month_

In [ ]:
mo_ =  driver.find_elements(By.XPATH,f'//td[@class="dxeCalendarFastNavMonth"] | //td[@class="dxeCalendarFastNavMonth dxeCalendarFastNavMonthSelected"] | //td[@class="dxeCalendarFastNavMonth dxeCalendarFastNavMonthSelected dxeCalendarFastNavMonthHover"]')
for e in mo_:
    print(e.text, type(e.text))

In [ ]:
month_selector = driver.find_elements(By.XPATH,f'//td[@class="dxeCalendarFastNavMonth"][text()={month_}] | //td[@class="dxeCalendarFastNavMonth dxeCalendarFastNavMonthSelected"][text()={month_}]')
for e in month_selector:
    print(e)

In [ ]:
month_selector = driver.find_elements(By.XPATH,f'//td[@class="dxeCalendarFastNavMonth"][text()="{month_}"] | //td[@class="dxeCalendarFastNavMonth dxeCalendarFastNavMonthSelected"][text()="{month_}"] | //td[@class="dxeCalendarFastNavMonth dxeCalendarFastNavMonthSelected dxeCalendarFastNavMonthHover"][text()="{month_}"]')

In [ ]:
# select the reporting month on the calendar
month_selector = driver.find_element(By.XPATH,f'//td[@class="dxeCalendarFastNavMonth"][text()="{month_}"] | //td[@class="dxeCalendarFastNavMonth dxeCalendarFastNavMonthSelected"][text()="{month_}"] | //td[@class="dxeCalendarFastNavMonth dxeCalendarFastNavMonthSelected dxeCalendarFastNavMonthHover"][text()="{month_}"]')
month_selector.click()

In [ ]:
# click the 'Submit' button
submit_button  = driver.find_element(By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_RunBtn"]')
submit_button.click()

In [ ]:
# enter fund names
name_      = driver.find_element(By.CSS_SELECTOR, 'input[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_FilterValue"]')
#name_tick_ = driver.find_element(By.CSS_SELECTOR, 'span[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SimpleFilterGrid_DXSelBtn0_D"]')
#name_tick_ = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SimpleFilterGrid_DXSelBtn0_D"]'))).click()

#name_.send_keys('LAUBAl')
#name_tick_.click()

for name in fnds_:
    name_.send_keys(name)
    #driver.execute_script(name, name_)
    wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'span[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SimpleFilterGrid_DXSelBtn0_D"]'))).click()
    name_.clear()

In [ ]:
for handle in driver.window_handles:
    driver.switch_to.window(handle)
    print(driver.title)

In [ ]:
x = driver.getWindowHandle[0]
x

In [ ]:
window_before = driver.window_handles[0]
window_before

In [ ]:
fund_selector.get_attribute("value")

In [ ]:
driver.find_element(By.CSS_SELECTOR, 'tr[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SelectedItemsGrid_DXSyncInput"][dxo.keys]').text

In [ ]:
driver.find_element(By.XPATH, "//*[@id='dxss_1044391392']//*[dxo.keys=['IJGIPF','PMMF','QIFFGIF']]")

#//*[@id="dxss_1044391392"]/text()

In [ ]:
# check that funds list is accurate
len(fnds_), len(fund_selector.get_attribute("value"))

In [ ]:
#fx = driver.find_element(By.CSS_SELECTOR, 'tr[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SelectedItemsGrid_DXDataRow3"]')
fx = driver.find_element(By.CSS_SELECTOR, 'tr[id="ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SelectedItemsGrid_DXSyncInput"]')


# assign values to the fund list elements
driver.execute_script(f'arguments[0].value = "IJGIPF, QIFFGIF";', fund_selector)
#ctl00_c_qc_QueryInputs_QueryInputsPopup_FUND0_SelectedItemsGrid_DXDataRow0